In [8]:
import requests
import re
import random
from bs4 import BeautifulSoup
import time

## list of random user agents

In [9]:
user_agents = [
    'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1',
    'Mozilla/5.0 (Linux; U; Android 4.4.2; en-us; SCH-I535 Build/KOT49H) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30',
    'Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows Phone 8.0; Trident/6.0; IEMobile/10.0; ARM; Touch; NOKIA; Lumia 920)',
    'Mozilla/5.0 (Linux; Android 7.0; SAMSUNG SM-G955U Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/5.4 Chrome/51.0.2704.106 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; U; Android 2.2; en-gb; Nexus One Build/FRF50) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
    'Mozilla/5.0 (Linux; Android 7.0; SM-G930VC Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/58.0.3029.83 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 8.0.0; SM-G960F Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.84 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 7.0; SM-G892A Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/60.0.3112.107 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 7.0; SM-G930VC Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/58.0.3029.83 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0.1; SM-G935S Build/MMB29K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0.1; SM-G920V Build/MMB29K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 7.1.1; G8231 Build/41.2.A.0.219; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/59.0.3071.125 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0.1; E6653 Build/32.2.A.0.253) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0; HTC One X10 Build/MRA58K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/61.0.3163.98 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0; HTC One M9 Build/MRA58K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.3',
    ]

## generate registration number and pac numbers

In [10]:
reg_number = str(random.randint(1000_0000,9999_9999))
pac_numbers = str(random.randint(10_000,99_999))
print(reg_number,'\n',pac_numbers)

89125619 
 86859


## scrape Irish IP addressses from this website: https://www.nirsoft.net/countryip/ie.html

In [11]:
res = requests.get('https://www.nirsoft.net/countryip/ie.html')
soup = BeautifulSoup(res.content,'lxml')
td_tags = soup.find_all('td') #217.183.255.255

scraped_ip_numbers = []

for td in td_tags:
    ip_address = re.match(r'\d\d\d.\d\d\d.\d\d\d.\d\d\d',td.text)
    if ip_address != None:
        scraped_ip_numbers.append(ip_address.group(0))

In [12]:
bogus_ip_addresses = []

for ip_address in scraped_ip_numbers:
    random_ip_address = ''
    for three_digits in ip_address.split('.'):
        randomized_ip_address = int(three_digits) + random.randint(-20,20)
        random_ip_address += str(randomized_ip_address)+'.'
    bogus_ip_addresses.append(random_ip_address[:-1])

['111.106.131.268',
 '99.268.272.252',
 '140.259.99.271',
 '140.201.245.275',
 '149.225.267.258',
 '140.218.257.255',
 '156.189.257.250',
 '143.191.133.260',
 '134.204.189.274',
 '140.210.222.242',
 '122.183.241.273',
 '159.209.274.263',
 '126.239.252.275',
 '136.250.242.236',
 '130.163.260.273',
 '139.162.270.255',
 '164.208.275.236',
 '167.154.132.244',
 '162.143.267.244',
 '170.172.248.264',
 '187.148.128.261',
 '171.169.158.267',
 '177.109.251.235',
 '187.169.116.266',
 '178.216.170.270',
 '187.97.163.264',
 '207.124.143.251',
 '186.128.142.237',
 '199.119.204.267',
 '210.130.234.243',
 '188.157.171.243',
 '201.118.141.247',
 '192.158.256.267',
 '214.168.249.237',
 '218.178.141.264',
 '199.176.275.273',
 '208.214.209.267',
 '228.244.143.270',
 '224.124.165.245',
 '223.104.136.237',
 '221.153.102.263',
 '208.184.225.246',
 '200.200.250.270']

## send the crap data to the scammers

In [13]:
def createRandomFormData(i):
    user_agent = user_agents[i%len(user_agents)]
    ip_address = bogus_ip_addresses[2*i%len(bogus_ip_addresses)]
    reg_number = str(random.randint(1000_0000,9999_9999))
    pac_numbers = str(random.randint(10_000,99_999))
    form_data = {
        'ip': ip_address ,
        'ua': user_agent,
        'regNumber': reg_number,
        'PACNumber': pac_numbers
        }
    return form_data

In [14]:
post_url = 'https://aib.ie1.online/files/action.php?type=login'

for i in range(1,4000):
    form_data = createRandomFormData(random.randint(1,1_000_000))
    post = requests.post(post_url,data = form_data)
    print('request number:',i,'form data: ',form_data,'status_code:',post.status_code)
    time.sleep(2)

request number: 1 form data:  {'ip': '186.128.142.237', 'ua': 'Mozilla/5.0 (Linux; Android 7.0; SM-G892A Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/60.0.3112.107 Mobile Safari/537.36', 'regNumber': '87402865', 'PACNumber': '94304'} status_code: 200
request number: 2 form data:  {'ip': '143.191.133.260', 'ua': 'Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36', 'regNumber': '21719656', 'PACNumber': '97783'} status_code: 200
request number: 3 form data:  {'ip': '99.268.272.252', 'ua': 'Mozilla/5.0 (Linux; Android 6.0.1; SM-G935S Build/MMB29K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36', 'regNumber': '50150392', 'PACNumber': '43081'} status_code: 200
request number: 4 form data:  {'ip': '199.119.204.267', 'ua': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.